In [ ]:
#protein sugar postive
import pandas as pd 
import os 
def get_B_sugar_biaowei(pdb_file_path,sugar_name,residue,protein_B_chain,protein_A_chain):
    B_count=0
    A_sugar_count=0
    B_line=[]
    A_sugar_line=[]
    #car_path_all='/data5_large/home/yanzh/antibody_car/test_car_protein/biaowei2/5A/protein_car/antibody_car/car_all'
    #car_path_l='/data5_large/home/yanzh/antibody_car/test_car_protein/biaowei2/5A/protein_car/antibody_car/car'
    with open(pdb_file_path,'r') as file:
        line = file.readline()
        B_list=[]
        A_sugar_list=[]
        #goon = False
        while line:
            dat_in = line[0:80].split()
            if (dat_in[0] == 'ATOM'):
                chain_id = line[21]
                #residue_id = int((line[22:26]))  
                x = float(line[30:38])
                y = float(line[38:46])
                z = float(line[46:54])
                atom_type2=line[77]
                residue_type = line[17:20]
                residue_id2 = str(line[22:27]).strip()  
                    #print(residue_type)
                    # First try CA distance of contact map
                atom_type = line[13:16].strip()
                if chain_id==protein_B_chain:   #蛋白B
                    if residue_type in residue:
                        if atom_type2 != 'H' :
                            B_list.append([x,y,z,atom_type,B_count,residue_id2,residue_type,chain_id,atom_type2])
                            B_line.append(line)
                            B_count += 1 

            elif  (line[0:6] == 'HETATM'):
                chain_id = line[21]
                #residue_id = int(line[22:26])        
                x = float(line[30:38])
                y = float(line[38:46])
                z = float(line[46:54])
                #Hetatm = line[0:5]
                residue_id2 = str(line[22:27]).strip() 
                atom_type2=line[77]
                residue_type = line[17:20]
                    #print(residue_type)
                    # First try CA distance of contact map
                atom_type = line[13:16].strip()
                if chain_id == protein_A_chain:
                    if residue_type in sugar_name:
                        if atom_type2 != 'H' :
                            A_sugar_list.append([x,y,z,atom_type,A_sugar_count,residue_id2,residue_type,chain_id,atom_type2])
                            A_sugar_line.append(line)
                            A_sugar_count += 1
            elif line.startswith('TER'):
                pass
            line = file.readline()
    #print(chain_car)
    #compute_1(r_list,l_list,receptor_list,sugar_list,count_car,path_r,pdb,protein_chain)  
    return(B_list,B_line,A_sugar_list,A_sugar_line,A_sugar_count)    

def compute_1(B_list,A_sugar_list,A_sugar_count):
    atom_len1=len(B_list)  #蛋白
    atom_len2=len(A_sugar_list)  #糖
    cut_off=5
    cut_off=cut_off**2
    #min_distance=1000000
    rog1 = []
    #chain_car=[]
    #rog2 = []
    if A_sugar_count !=0:
        for i in range(atom_len1):  #蛋白
            atom1=B_list[i]
            ab_chain=atom1[7]
            #ab_atom_name=atom1[3]
            ab_residue_id=atom1[5]
            #ab_residue_type=atom1[6]
            #ab_atom=atom1[8]
            for j in range(atom_len2):  #糖
                atom2=A_sugar_list[j]
                #car_chain=atom2[7]
                #car_type=atom2[6]
                #car_id=atom2[5]
                #car_atom_name=atom2[3]
                #car_atom=atom2[8]
                distance = 0
                for k in range(3):
                    distance+=(atom1[k]-atom2[k])**2
                if distance<=cut_off:
                    rog1.append((ab_chain,ab_residue_id))
                    #if car_chain not in chain_car:
                    #    chain_car.append(car_chain)
                    #rog2.append((car_chain,car_id))
    rog1=list(set(rog1))
    #rog2=list(set(rog2))
    return(rog1)    

def Form_interface(rog,B_line,path_r,pdb,protein_B_chain,protein_A_chain):
    #ligand=Form_interface(receptor_list,ligand_list,rog1,rog2)
    #print(ligand)
    final_receptor=[]
    for i in B_line:   #line逐行运行
        chain_id = i[21]
        residue_id2 = str(i[22:27]).strip()  
        for j in rog:  
            ch=j[0]
            re=j[1]
            if chain_id == ch and residue_id2 == re:
                final_receptor.append(i)
    Write_Interface(final_receptor,path_r,"%s_%s_%s_%s.pdb"%(pdb,protein_B_chain,protein_A_chain,protein_B_chain))

def Write_Interface(final_atom,path,name):
    if not final_atom:
        return None
    new_path=os.path.join(path,name)
    with open(new_path,'w') as file:
        for line in final_atom:
            file.write(line)   

data=pd.read_csv('/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new/dataset_shuanglian/interaction_B_and_carprotein_A.csv')
sugar_name= ['A2G','B6D','BGC','BMA','FCA','FUC','FUL','GAL','GLA','GLC','LXB','LXZ','M6D','MAN','NAG','NDG','NGA','SHG','SIA','XYP','XYS']
path_r='/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new/dataset_shuanglian/proB/protein_only_car_postive'
residue= ['CYS','ASP','SER','GLN','LYS','ILE','PRO','THR','PHE','ASN','GLY','HIS','LEU','ARG','TRP','ALA', 'VAL','GLU','TYR',
                            'MET','THC','CYF', 'DVA','DPR','DSN','DGN','DHI','DTY','DAS','AIB','ORQ','YCM','PCA','ACE','PTR','SEP','TPO','M3L','TYS']
for idx,value in data.iterrows():
    pdb2=value['pdb_name']
        #if pdb2 !='4BSE_1_A':
        #    continue
    pdb=value['pdb_name'][0:6]
        #if pdb2 in pdbname:
    print(pdb2)
    protein_B_chain=value['protein_B_chain']
    protein_A_chain=value['sugar_protein_A_chain']
    pdb_file_path=f'/data5_large/home/yanzh/deeplearn/PeSTo/data_my/gz_interaction_structure2/updata/{pdb}.pdb'
    B_list,B_line,A_sugar_list,A_sugar_line,A_sugar_count=get_B_sugar_biaowei(pdb_file_path,sugar_name,residue,protein_B_chain,protein_A_chain)
    rog1=compute_1(B_list,A_sugar_list,A_sugar_count)  
    Form_interface(rog1,B_line,path_r,pdb,protein_B_chain,protein_A_chain)